DU & DP

Preamble

In [1]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

!pip install scikeras
from scikeras.wrappers import KerasClassifier

In [2]:
# load dataset
dataset = numpy.loadtxt("5A pima-indians-diabetes.data.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
y = dataset[:,8]

print(X)


[[  6.    148.     72.    ...  33.6     0.627  50.   ]
 [  1.     85.     66.    ...  26.6     0.351  31.   ]
 [  8.    183.     64.    ...  23.3     0.672  32.   ]
 ...
 [  5.    121.     72.    ...  26.2     0.245  30.   ]
 [  1.    126.     60.    ...  30.1     0.349  47.   ]
 [  1.     93.     70.    ...  30.4     0.315  23.   ]]


Define grid functions for KerasClassifier

In [3]:
def create_and_compile_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))

	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model


Create model

In [4]:
# create model
model = KerasClassifier(build_fn=create_and_compile_model, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]

param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring="accuracy", n_jobs=-1, cv=3)

grid_result = grid.fit(X, y)


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Evaluate output

In [5]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']

params = grid_result.cv_results_['params']

for mean, param in zip(means, params):
    print("train accuracy %f for: %r" % (mean, param))

Best: 0.708333 using {'batch_size': 10, 'epochs': 100}
train accuracy 0.613281 for: {'batch_size': 10, 'epochs': 10}
train accuracy 0.640625 for: {'batch_size': 10, 'epochs': 50}
train accuracy 0.708333 for: {'batch_size': 10, 'epochs': 100}
train accuracy 0.555990 for: {'batch_size': 20, 'epochs': 10}
train accuracy 0.684896 for: {'batch_size': 20, 'epochs': 50}
train accuracy 0.684896 for: {'batch_size': 20, 'epochs': 100}
train accuracy 0.588542 for: {'batch_size': 40, 'epochs': 10}
train accuracy 0.634115 for: {'batch_size': 40, 'epochs': 50}
train accuracy 0.673177 for: {'batch_size': 40, 'epochs': 100}
train accuracy 0.498698 for: {'batch_size': 60, 'epochs': 10}
train accuracy 0.596354 for: {'batch_size': 60, 'epochs': 50}
train accuracy 0.656250 for: {'batch_size': 60, 'epochs': 100}
train accuracy 0.466146 for: {'batch_size': 80, 'epochs': 10}
train accuracy 0.647135 for: {'batch_size': 80, 'epochs': 50}
train accuracy 0.700521 for: {'batch_size': 80, 'epochs': 100}
train accu

Verify best hyperparameters

In [6]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X, y, epochs=100, batch_size=60)

# evaluate the keras model
_, accuracy = model.evaluate(X, y)

print('\n Final Accuracy: %.2f' % (accuracy*100))

Epoch 1/100
13/13 [==============================] - 0s 1ms/step - loss: 2.1219 - accuracy: 0.5117
Epoch 2/100
13/13 [==============================] - 0s 2ms/step - loss: 1.7821 - accuracy: 0.5182
Epoch 3/100
13/13 [==============================] - 0s 3ms/step - loss: 1.5358 - accuracy: 0.5404
Epoch 4/100
13/13 [==============================] - 0s 2ms/step - loss: 1.3542 - accuracy: 0.5534
Epoch 5/100
13/13 [==============================] - 0s 2ms/step - loss: 1.2018 - accuracy: 0.5677
Epoch 6/100
13/13 [==============================] - 0s 2ms/step - loss: 1.1522 - accuracy: 0.5938
Epoch 7/100
13/13 [==============================] - 0s 6ms/step - loss: 1.0752 - accuracy: 0.6003
Epoch 8/100
13/13 [==============================] - 0s 5ms/step - loss: 1.0227 - accuracy: 0.6094
Epoch 9/100
13/13 [==============================] - 0s 3ms/step - loss: 1.0155 - accuracy: 0.6146
Epoch 10/100
13/13 [==============================] - 0s 6ms/step - loss: 0.9496 - accuracy: 0.6094
Epoch 11/

###Sweep number of neurons and number of layers

Re-define model creation function

In [7]:
def create_and_compile_model(num_neurons, num_layers):
	# create model
  model = Sequential()
  model.add(Dense(num_neurons, input_dim=8, activation='relu'))

  if num_layers > 0:

    while num_layers > 0:
      model.add(Dense(num_neurons, activation='relu'))
      num_layers=num_layers-1


  model.add(Dense(1, activation='sigmoid')) # output layer

  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  print(model.summary())
  print('--')

  return model


Create new model

In [8]:
# create model
model = KerasClassifier(build_fn=create_and_compile_model,
                        epochs=100,
                        batch_size=10,
                        verbose=0,
                        num_layers=0,
                        num_neurons = 3)

# define the grid search parameters
num_neurons_grid = [3, 100]
num_layers_grid = [0, 4]

param_grid = dict(num_neurons=num_neurons_grid, num_layers=num_layers_grid)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

grid_result = grid.fit(X, y)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               900       
                                                                 
 dense_5 (Dense)             (None, 100)               10100     
                                                                 
 dense_6 (Dense)             (None, 100)               10100     
                                                                 
 dense_7 (Dense)             (None, 100)               10100     
                                                                 
 dense_8 (Dense)             (None, 100)               10100     
                                                                 
 dense_9 (Dense)             (None, 1)                 101       
                                                                 
Total params: 41401 (161.72 KB)
Trainable params: 4140

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


None
--


Evaluate output

In [9]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']

params = grid_result.cv_results_['params']

for mean, param in zip(means, params):
    print("train accuracy %f for: %r" % (mean, param))

Best: 0.731771 using {'num_layers': 4, 'num_neurons': 100}
train accuracy 0.664062 for: {'num_layers': 0, 'num_neurons': 3}
train accuracy 0.709635 for: {'num_layers': 0, 'num_neurons': 100}
train accuracy 0.680990 for: {'num_layers': 4, 'num_neurons': 3}
train accuracy 0.731771 for: {'num_layers': 4, 'num_neurons': 100}


Verify best hyperparameters

In [10]:
# define the keras model
model2 = Sequential()
model2.add(Dense(100, input_dim=8, activation='relu'))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

# compile the keras model
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model2.fit(X, y, epochs=100, batch_size=10)

# evaluate the keras model
_, accuracy = model2.evaluate(X, y)

print('\n Final Accuracy: %.2f' % (accuracy*100))

Epoch 1/100
77/77 [==============================] - 1s 1ms/step - loss: 1.0431 - accuracy: 0.5977
Epoch 2/100
77/77 [==============================] - 0s 1ms/step - loss: 0.6840 - accuracy: 0.6641
Epoch 3/100
77/77 [==============================] - 0s 1ms/step - loss: 0.6597 - accuracy: 0.6719
Epoch 4/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6503 - accuracy: 0.6615
Epoch 5/100
77/77 [==============================] - 0s 1ms/step - loss: 0.6027 - accuracy: 0.6888
Epoch 6/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6336 - accuracy: 0.7031
Epoch 7/100
77/77 [==============================] - 0s 1ms/step - loss: 0.6169 - accuracy: 0.6628
Epoch 8/100
77/77 [==============================] - 0s 1ms/step - loss: 0.5744 - accuracy: 0.7044
Epoch 9/100
77/77 [==============================] - 0s 1ms/step - loss: 0.5721 - accuracy: 0.7096
Epoch 10/100
77/77 [==============================] - 0s 2ms/step - loss: 0.5606 - accuracy: 0.7253
Epoch 11/

This demo is modified from https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/